In [1]:
## DO NOT ERASE THIS. IMPORTANT TO CORRECTLY IMPORT MODULES
import sys

sys.path.append("../")
sys.executable

'/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/bin/python3.12'

In [2]:
import pandas as pd

pd.set_option("display.max_rows", 102)
from tqdm.auto import tqdm

import torch
from torch.utils.data import DataLoader, TensorDataset

from src.utils import (
    load_MovieLens,
    create_test_user,
    train_test_val_split,
    seed_everything,
    UserMovieDataset,
    split_test_df,
    add_not_watched_movies,
    create_test_user_display_df,
    train,
    predict,
)
from src.models import NeuMF
from src.metrics import reccomendation_report

## Constants

In [3]:
DATA_FOLDER = "../../data/ml-1m/"
RANDOM_STATE = 7

In [4]:
seed_everything(RANDOM_STATE)

## Data

In [5]:
df_users, df_movies, df_ratings = load_MovieLens(DATA_FOLDER)

### Creating a test_user

In [6]:
df_users, df_ratings, new_user_id = create_test_user(
    df_users, df_ratings, [6, 16, 3192, 1461, 827, 887, 593]
)
print("Test user watch list:")
display(df_ratings[df_ratings.userId == new_user_id].merge(df_movies, on="movieId"))

Test user watch list:


,userId,movieId,rating,timestamp,name,genre
0,6040,6,5,0,Sabrina (1995),Comedy|Romance
1,6040,16,5,1,Sense and Sensibility (1995),Drama|Romance
2,6040,3192,5,2,Singles (1992),Comedy|Drama|Romance
3,6040,1461,5,3,Love and Other Catastrophes (1996),Romance
4,6040,827,5,4,Emma (1996),Comedy|Drama|Romance
5,6040,887,5,5,Singin' in the Rain (1952),Musical|Romance
6,6040,593,5,6,Pretty Woman (1990),Comedy|Romance


### Train-test split
Methodology: Last user interaction is a test item. The rest is train. Validation part is 20% of test.

In [7]:
df_train, df_test, df_val = train_test_val_split(df_ratings, df_movies, RANDOM_STATE)

Enriching test:   0%|          | 0/4832 [00:00<?, ?it/s]

Enriching val:   0%|          | 0/1209 [00:00<?, ?it/s]

### Loading additional data

In [8]:
# cos_dist = computeCosineSimilarities(df_train, "userId", "movieId", df_movies["movieId"].nunique())

# with open("../../data/cos_dist.pt", "wb") as f:
#     torch.save(cos_dist, f)

with open("../../data/cos_dist.pt", "rb") as f:
    cos_dist = torch.load(f)
popularity = torch.bincount(
    torch.tensor(df_train.movieId.values), minlength=df_movies["movieId"].nunique()
)
popularity = popularity / torch.max(popularity)

## NeuMF Best

In [9]:
seed_everything(RANDOM_STATE)
train_loader = DataLoader(
    UserMovieDataset(df_train, df_movies["movieId"].nunique()),
    batch_size=2048,
    shuffle=True,
)
val_loader = DataLoader(
    TensorDataset(torch.tensor(df_val["userId"]), torch.tensor(df_val["movieId"])),
    batch_size=4096,
    shuffle=False,
)
test_loader = DataLoader(
    TensorDataset(torch.tensor(df_test["userId"]), torch.tensor(df_test["movieId"])),
    batch_size=4096,
    shuffle=False,
)
num_users = df_users["userId"].nunique()
num_items = df_movies["movieId"].nunique()

  0%|          | 0/994175 [00:00<?, ?it/s]

**Best hyper parametres combination after tuning:**

Trial 3 finished with value: 0.6578366160392761 and parameters: {

'mf_dim': 32.0, 

'n_mlp_layers': 6, 

'mlp_layers_dim': 496.0, 

'mlp_kwargs_dropout': False, 

'mlp_kwargs_batchnorm': True, 

'mlp_kwargs_dropout_rate': 0.4536392190441475, 

'lr': 0.004144556029028259, 

'n_epochs': 7.0

}

In [10]:
seed_everything(RANDOM_STATE)
neumf = NeuMF(
    num_users,
    num_items,
    mf_dim=32,
    n_mlp_layers=6,
    mlp_layers_dim=396,
    mlp_kwargs={
        "activation": True,
        "dropout": False,
        "batchnorm": True,
        "dropout_rate": 0.45,
    },
)
display(neumf)

NeuMF(
  (mf_user_embed): Embedding(6041, 32)
  (mf_item_embed): Embedding(3883, 32)
  (mlp_user_embed): Embedding(6041, 198)
  (mlp_item_embed): Embedding(3883, 198)
  (mlp): Sequential(
    (MLP_layer_0): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=396, out_features=396, bias=True)
        (Activation): ReLU()
        (BatchNorm): BatchNorm1d(396, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (MLP_layer_1): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=396, out_features=396, bias=True)
        (Activation): ReLU()
        (BatchNorm): BatchNorm1d(396, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (MLP_layer_2): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=396, out_features=396, bias=True)
        (Activation): ReLU()
        (BatchNorm): BatchNorm1d(396, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (MLP_la

In [11]:
optimizer = torch.optim.Adam(neumf.parameters(), lr=0.0041)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)
criterion = torch.nn.BCELoss()
n_epochs = 7
device = "cpu"

In [12]:
%%time
train(
    neumf,
    train_loader,
    optimizer,
    scheduler,
    criterion,
    n_epochs,
    val_loader=val_loader,
    df_val=df_val,
    cos_dist=cos_dist,
    popularity=popularity,
    verbose=True,
)

Epochs:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/2428 [00:00<?, ?batch/s]

Epoch: 0
Train loss: 0.36059


Inference:   0%|          | 0/30 [00:00<?, ?batch/s]

{'Hit rate @ 15': tensor(0.5997),
 'NDCG @ 15': tensor(0.2855),
 'Diversity (ILD)': tensor(0.1366),
 'Novelty (EPC)': tensor(0.7046)}


...

{'Hit rate @ 15': tensor(0.7229),
 'NDCG @ 15': tensor(0.3756),
 'Diversity (ILD)': tensor(0.1087),
 'Novelty (EPC)': tensor(0.7752)}


  0%|          | 0/2428 [00:00<?, ?batch/s]

Epoch: 6
Train loss: 0.21478


Inference:   0%|          | 0/30 [00:00<?, ?batch/s]

{'Hit rate @ 15': tensor(0.7196),
 'NDCG @ 15': tensor(0.3710),
 'Diversity (ILD)': tensor(0.1078),
 'Novelty (EPC)': tensor(0.7781)}
CPU times: user 38min 13s, sys: 1h 32min 7s, total: 2h 10min 21s
Wall time: 13min 58s


In [13]:
df_test["pred"] = predict(neumf, test_loader, verbose=False).numpy()
pred, target, pred_items = split_test_df(df_test, "userId", "movieId", "pred", "action")
reccomendation_report(pred, target, pred_items, cos_dist, popularity, k=15)

{'Hit rate @ 15': tensor(0.7221),
 'NDCG @ 15': tensor(0.3752),
 'Diversity (ILD)': tensor(0.1080),
 'Novelty (EPC)': tensor(0.7779)}

## Test user recommendations

In [14]:
df_test_user = add_not_watched_movies(new_user_id, df_test, df_train, df_movies)
test_user_loader = DataLoader(
    TensorDataset(
        torch.tensor(df_test_user["userId"]), torch.tensor(df_test_user["movieId"])
    ),
    batch_size=4096,
    shuffle=False,
)
df_test_user["pred"] = predict(neumf, test_user_loader, verbose=False).numpy()
display(create_test_user_display_df(df_test_user, df_movies, "pred"))

,userId,movieId,name,genre,pred
0,6040,10,"American President, The (1995)",Comedy|Drama|Romance,0.970429
1,6040,1171,Strictly Ballroom (1992),Comedy|Romance,0.941488
2,6040,219,Circle of Friends (1995),Drama|Romance,0.927331
3,6040,1393,Evita (1996),Drama|Musical,0.910708
4,6040,464,"Englishman Who Went Up a Hill, But Came Down a...",Comedy|Romance,0.909639
5,6040,1656,Good Will Hunting (1997),Drama,0.909190
6,6040,1720,"Wedding Singer, The (1998)",Comedy|Romance,0.907276
7,6040,1081,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi,0.887048
8,6040,2252,Pleasantville (1998),Comedy,0.873411
9,6040,139,"Birdcage, The (1996)",Comedy,0.869268
